# Lab #2: Randomized Gradient Methods

by Peter Richtarik

## The Problem: Ridge Regression

In this lab we will experiment with the NSync [1] algorithm applied to the ridge regression problem:

$$\min_{x\in \mathbb{R}^n} \left[ f(x) = \frac{1}{2}\|Ax-b\|_2^2 + \frac{\lambda}{2} \|x\|_2^2\right],$$

where $A$ is an $m\times n$ matrix and $b\in \mathbb{R}^n$. Recall that the gradient of $f$ is given by

$$\nabla f(x) =  A^T (Ax-b) + \lambda x.$$

[1] Richtarik and Takac. On optimal probabilities in stochastic coordinate descent methods. Optimization Letters, 2015.

## NSync Algorithm

The NSync algorithm proceeds as follows:

- Initialization: 
  - Start with $x_0\in \mathbb{R}^n$
  - Fix a proper sampling $\hat{S}$ and compute positive parameters $v = (v_1,\dots,v_n)$ such that 

$$ (f,\hat{S}) \sim ESO(v)$$

- In iteration $k$ do:
  - Draw a fresh copy of $S_k\sim \hat{S}$ (i.e., independently from everything else)
  - For $i\in S_k$ do
      $$ x_{k+1}^{(i)} = x_k^{(i)} - \frac{1}{v_i} e_i^T \nabla f(x_k) $$
  - For $i\notin S_k$ do
      $$ x_{k+1}^{(i)} = x_k^{(i)}$$

Recall that a sampling $\hat{S}$ is proper if the quantity $p_i = \mathbf{Prob}(i\in \hat{S})$ is positve for all $i\in [n]=\{1,2,\dots,n\}$.

## Computing ESO parameters 

In order to be able to run the NSync method, we need to pre-compute the ESO parameters $v_1,\dots,v_n$ before the methods starts. The following result will be helpful in this regard:

${\bf Theorem}$ Let $P = (p_{ij})\in \mathbb{R}^{n\times n}$ be the probability matrix associated with sampling $\hat{S}$. That is,  $p_{ij} = \mathbf{Prob}(i\in \hat{S}, j\in \hat{S})$. Then $(f,\hat{S})\sim ESO(v)$ holds if $v$ satisfies the inequality

$$ P \bullet (A^TA + \lambda I) \preceq Diag(p_1 v_1, \dots, p_n v_n),$$

where $p_i = p_{ii}$.

We have the following useful consequence:

${\bf Corollary}$ If $\hat{S}$ is a serial sampling, i.e., if $p_{ij} = 0$ whenever $i\neq j$, then $P$ is diagonal and so 

$$P \bullet (A^TA + \lambda I) = Diag(p_1 (\|A_{:1}\|_2^2 + \lambda), \dots, p_n (\|A_{:n}\|_2^2 + \lambda)).$$

Therefore, $(f,\hat{S})\sim ESO(v)$ with 

$$v_i = \|A_{:i}\|_2^2 + \lambda, \qquad i=1,\dots,n.$$

## 1. Data

In [53]:
m = 10
n = 1000
A = sprandn(m,n,0.1)/m # sparse random matrix
b = ones(m,1)
lambda = 1

x_star = (A'*A + lambda*eye(n))\(A'*b) # finds the point x where derivative is zero (the solution)

1000x1 Array{Float64,2}:
  0.0116559
  0.0      
  0.0      
  0.0      
  0.115493 
 -0.214575 
  0.0766919
  0.0104832
 -0.0666041
  0.0      
 -0.0320028
 -0.0290425
 -0.0933021
  ⋮        
  0.0127912
 -0.014639 
  0.0      
 -0.0180058
 -0.0493315
  0.0      
  0.0124382
  0.0      
  0.0138768
  0.0447612
  0.0154187
  0.0      

## 2. NSync

In [48]:
function NSync(A, b, lambda, T)
    
    m, n = size(A)
    
    v = zeros(n,1)
    x = zeros(n,1)    # staring point
    g = -b            # g will be maintained to be equal to A*x - b
    
    for i=1:n         # compute ESO parameters
        v[i] = (norm(A[:,i]))^2 + lambda
    end
    
    for k = 1:T
        i = rand(1:n)                                   # choose a random index 
        h = (1/v[i]) * ( A[:,i]' * g + lambda * x[i] )  # compute the update
        x[i] = x[i] - h[1]                              # perform the update 
        g = g - h[1] * A[:,i]                           # update the residual
    end
    
    return x
    
end

NSync (generic function with 1 method)

## 3. Testing

In [54]:
T = 50*n
x = NSync(A,b,lambda,T)
norm(x-x_star)

7.067384249535481e-15

## Problem A

Write a new version of the NSync function and call it NSyncPlot. This function will also visualize the results via a plot, as in Lab 1. Copy paste the relevant code from Lab 1 and modify to make it work.

## Problem B

Implement NSync for 2 different serial samplings and compare the results.

## Problem C

Implement NSync with the deterministic sampling for which $\hat{S}=\{1,2,\dots,n\}$ with probability 1. NSync transforms into standard gradient descent. Compute the parameters $v$ satisfying the ESO assumption, and run the method for 50 iterations. Compare with NSync which uses the serial sampling and which is run for $T = 50n$ iterations. Is this comparison fair? 


## Problem D

Implement a version of NSync for which $|\hat{S}| = 100$ with probability 1. Compare with the previously developed versions.

## Problem E

Find a way to perform the for loop in NSync in parallel. You will need to study Julia's documentation to pull this off.

## Problem F

Write a new version of NSync (with any sampling you like), this time applied to logistic regression. 